In [143]:
import pandas as pd
import numpy as np
from google.colab import drive
from scipy.stats import randint
from google.colab import files

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import Ridge
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import LinearSVC
from sklearn.linear_model import SGDClassifier

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

from sklearn.feature_selection import SelectFromModel
from sklearn.feature_selection import RFE
from sklearn.feature_selection import RFECV

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

In [144]:
df = pd.read_csv('/train.csv');
df.shape

(891, 12)

In [145]:
# Drop null values
data = df
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Pclass       891 non-null    int64  
 2   Name         891 non-null    object 
 3   Sex          891 non-null    object 
 4   Age          714 non-null    float64
 5   SibSp        891 non-null    int64  
 6   Parch        891 non-null    int64  
 7   Ticket       891 non-null    object 
 8   Fare         891 non-null    float64
 9   Cabin        204 non-null    object 
 10  Embarked     889 non-null    object 
 11  Survived     891 non-null    int64  
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [146]:
data['Cabin'].fillna("UnKnown", inplace=True)

In [147]:
data['Age'].fillna(29.7, inplace=True)

In [148]:
def substrings_in_string(big_string, substrings):
    for substring in substrings:
        if big_string.find(substring) != -1:
            return substring
    print (big_string)
    return np.nan

In [149]:
title_list=['Mrs', 'Mr', 'Master', 'Miss', 'Major', 'Rev',
                    'Dr', 'Ms', 'Mlle','Col', 'Capt', 'Mme', 'Countess',
                    'Don', 'Jonkheer']

In [150]:
data['Title']=data['Name'].map(lambda x: substrings_in_string(x, title_list))
#replacing all titles with mr, mrs, miss, master
def replace_titles(x):
    title=x['Title']
    if title in ['Don', 'Major', 'Capt', 'Jonkheer', 'Rev', 'Col']:
        return 'Mr'
    elif title in ['Countess', 'Mme']:
        return 'Mrs'
    elif title in ['Mlle', 'Ms']:
        return 'Miss'
    elif title =='Dr':
        if x['Sex']=='Male':
            return 'Mr'
        else:
            return 'Mrs'
    else:
        return title
data['Title']=data.apply(replace_titles, axis=1)

In [151]:
data.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Survived,Title
0,1,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,UnKnown,S,0,Mr
1,2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,1,Mrs
2,3,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,UnKnown,S,1,Miss
3,4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,1,Mrs
4,5,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,UnKnown,S,0,Mr


In [152]:
cabin_list = ['A', 'B', 'C', 'D', 'E', 'F', 'T', 'G', 'UnKnown']
data['Deck']=data['Cabin'].map(lambda x: substrings_in_string(x, cabin_list))

In [153]:
#Creating new family_size column
data['Family_Size']=data['SibSp']+data['Parch']

In [154]:
data['Age*Class']=data['Age']*data['Pclass']

In [155]:
data['Fare_Per_Person']=data['Fare']/(data['Family_Size']+1)

In [156]:
data['Fare_Per_Person']=data['Fare']/(data['Family_Size']+1)

In [159]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 17 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   PassengerId      891 non-null    int64  
 1   Pclass           891 non-null    int64  
 2   Name             891 non-null    object 
 3   Sex              891 non-null    int64  
 4   Age              891 non-null    float64
 5   SibSp            891 non-null    int64  
 6   Parch            891 non-null    int64  
 7   Ticket           891 non-null    object 
 8   Fare             891 non-null    float64
 9   Cabin            891 non-null    object 
 10  Embarked         889 non-null    object 
 11  Survived         891 non-null    int64  
 12  Title            891 non-null    int64  
 13  Deck             891 non-null    int64  
 14  Family_Size      891 non-null    int64  
 15  Age*Class        891 non-null    float64
 16  Fare_Per_Person  891 non-null    float64
dtypes: float64(4), i

In [160]:
data = data.dropna()

In [161]:
# Assigning numerical values and storing in another column
from sklearn.preprocessing import LabelEncoder

labelencoder = LabelEncoder()
data['Title'] = labelencoder.fit_transform(data['Title'])
data['Deck'] = labelencoder.fit_transform(data['Deck'])
data['Sex'] = labelencoder.fit_transform(data['Sex'])
data['Embarked'] = labelencoder.fit_transform(data['Embarked'])


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-doc

In [170]:
data_no = data.copy().drop(['Survived', 'Name', 'Cabin', 'PassengerId', 'Ticket'], axis=1)
y = data['Survived']

In [169]:
data_no.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Title,Deck,Family_Size,Age*Class,Fare_Per_Person
0,3,1,22.0,1,0,7.2500,2,2,8,1,66.0,3.62500
1,1,0,38.0,1,0,71.2833,0,3,2,1,38.0,35.64165
2,3,0,26.0,0,0,7.9250,2,1,8,0,78.0,7.92500
3,1,0,35.0,1,0,53.1000,2,3,2,1,35.0,26.55000
4,3,1,35.0,0,0,8.0500,2,2,8,0,105.0,8.05000


In [189]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_validate
clf = DecisionTreeClassifier(random_state=0)
metrics = ['f1_macro','precision_macro', 'recall_macro']
scores = cross_validate(clf, data_no, y, cv=10, scoring=metrics)
print(sorted(scores.keys()))
scores
for i in scores:
  print(i)
  print(np.mean(scores[i]))
  # np.mean(i)

['fit_time', 'score_time', 'test_f1_macro', 'test_precision_macro', 'test_recall_macro']
fit_time
0.004982709884643555
score_time
0.0032560110092163088
test_f1_macro
0.7775082720984658
test_precision_macro
0.7803454794981085
test_recall_macro
0.7790829867300456


In [191]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
scores = cross_validate(gnb, data_no, y, cv=10, scoring=metrics)
print(sorted(scores.keys()))
scores
for i in scores:
  print(i)
  print(np.mean(scores[i]))

['fit_time', 'score_time', 'test_f1_macro', 'test_precision_macro', 'test_recall_macro']
fit_time
0.002548837661743164
score_time
0.003662586212158203
test_f1_macro
0.764742966692199
test_precision_macro
0.7746695448110736
test_recall_macro
0.7609952465834819


In [192]:
from sklearn.dummy import DummyClassifier
dc = DummyClassifier()
scores = cross_validate(dc, data_no, y, cv=10, scoring=metrics)
print(sorted(scores.keys()))
scores
for i in scores:
  print(i)
  print(np.mean(scores[i]))

['fit_time', 'score_time', 'test_f1_macro', 'test_precision_macro', 'test_recall_macro']
fit_time
0.0014121055603027344
score_time
0.0028460025787353516
test_f1_macro
0.4944208397069777
test_precision_macro
0.4956818562148989
test_recall_macro
0.4955189146365617


/usr/local/lib/python3.6/dist-packages/sklearn/dummy.py:132: FutureWarning: The default value of strategy will change from stratified to prior in 0.24.
  "stratified to prior in 0.24.", FutureWarning)


In [185]:
def feature_selection(data):
  data['Cabin'].fillna("UnKnown", inplace=True)
  data['Age'].fillna(29.7, inplace=True)
  data['Title']=data['Name'].map(lambda x: substrings_in_string(x, title_list))
  data['Title']=data.apply(replace_titles, axis=1)
  data['Deck']=data['Cabin'].map(lambda x: substrings_in_string(x, cabin_list))
  data['Family_Size']=data['SibSp']+data['Parch']
  data['Fare_Per_Person']=data['Fare']/(data['Family_Size']+1)
  data['Age*Class']=data['Age']*data['Pclass']

  data['Title'] = labelencoder.fit_transform(data['Title'])
  data['Deck'] = labelencoder.fit_transform(data['Deck'])
  data['Sex'] = labelencoder.fit_transform(data['Sex'])
  data['Embarked'] = labelencoder.fit_transform(data['Embarked'])

  data = data.drop(['Name', 'Cabin', 'PassengerId', 'Ticket'], axis=1)
  return data

In [179]:
def downloadTarget(target, fileName):
  submission = pd.DataFrame(target, columns=['prediction'])
  submission.to_csv(fileName) 
  files.download(fileName)

In [176]:
test = pd.read_csv('/test.csv');
test['Fare'].fillna(32.204, inplace=True)
test = feature_selection(test)
test.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Title,Deck,Family_Size,Fare_Per_Person,Age*Class
0,3,1,34.5,0,0,7.8292,1,2,7,0,7.829200,103.5
1,3,0,47.0,1,0,7.0000,2,3,7,1,3.500000,141.0
2,2,1,62.0,0,0,9.6875,1,2,7,0,9.687500,124.0
3,3,1,27.0,0,0,8.6625,2,2,7,0,8.662500,81.0
4,3,0,22.0,1,1,12.2875,2,3,7,2,4.095833,66.0


In [181]:
gnb = GaussianNB()
gnb.fit(data_no, y)
predicted = gnb.predict(test)
downloadTarget(predicted, 'naiveBayes.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [183]:
from sklearn.dummy import DummyClassifier
dc = DummyClassifier()
dc.fit(data_no, y)
predicted = dc.predict(test)
downloadTarget(predicted, 'dummyClassifier.csv')

/usr/local/lib/python3.6/dist-packages/sklearn/dummy.py:132: FutureWarning: The default value of strategy will change from stratified to prior in 0.24.
  "stratified to prior in 0.24.", FutureWarning)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
clf = DecisionTreeClassifier(random_state=0)
clf.fit(data_no, y)
predicted = clf.predict(test)
predicted.shape